In [1]:
guideline = (
    "Act as an ULTIMATE Python Coding Expert. Follow instructions given below STRICTLY.\n"
    "Input: A programming question in English with function name and parameters and test cases list.\n"
    "Output: Only the correct Python code inside a fenced code block.\n\n"

    "Format:\n"
    "```python\n"
    "<code here>\n"
    "```\n\n"

    "Here is example question:\n"
    """Instruction: Write a python program to find the sum of first n natural numbers.\n
    Example:\n
    def sum_series(number):\n
        #code here\n
        return total\n
    """
    "Output:\n"
    "```python\n"
    "import math\n"
    "def sum_series(number):\n"
    "    total = 0\n"
    "    total = (number * (number + 1)) / 2\n"
    "    return total\n"
    "```\n\n"

    "Rules:\n"
    "- DEEPLY think about all possible solutions step by step and ALWAYS provide the CORRECT code.\n"
    "- Handle all possible edge cases ROBUSTLY and the code MUST PASS all the test cases.\n"
    "- Preserve the function name and parameters exactly as provided.\n"
    "- Always use return statements to output results.\n"
    "- Write Python code without using classes. Only use functions and variables.\n"
    "- If the function needs imports, import required libraries.\n"
    "- Do not include any text, explanation, comments, or docstrings.\n"
    "- Do not write code for reading input (no input()) or printing output (no print()).\n"
)

In [7]:
!pip install -q anthropic pandas tqdm

In [ ]:
import anthropic

client = anthropic.Anthropic(api_key='YOUR_API_KEY')

In [9]:
!pip install -q transformers sentencepiece

## check if it works or not

In [10]:
bangla_prompt = """একটি ফাংশন লিখুন যা n এর চেয়ে ছোট বা সমান একটি স্বচ্ছ সংখ্যা পেতে পারে।
Exammple:
get_ludic(n)"""

# Define system prompt (guideline)
# This should be a **concise instruction about Claude’s behavior** — not the entire detailed rules.
system_prompt = "You are a strict Python code generator. Follow the user instruction exactly. Never include any explanation or extra text."


full_prompt = (
        f"{guideline}\n\n"
        f"Here is the programming question: {bangla_prompt}\n\n"
        f"Now write a Python code that will pass all of the hidden test cases."
    )

# Claude generation

response = client.messages.create(
    model="claude-sonnet-4-20250514",  # or use claude-3.5-sonnet-20240620
    max_tokens=512,
    temperature=0.3,
    system=system_prompt,
    messages=[
        {"role": "user", "content": full_prompt}
    ]
)

# Extract and display result
response_text = response.content[0].text.strip()
print("🧠 Claude Output:\n", response_text)

🧠 Claude Output:
 ```python
def get_ludic(n):
    if n < 1:
        return []
    
    sieve = list(range(1, n + 1))
    ludic = []
    
    while sieve:
        ludic.append(sieve[0])
        if len(sieve) == 1:
            break
        
        step = sieve[0]
        new_sieve = []
        
        for i in range(len(sieve)):
            if (i + 1) % step != 0:
                new_sieve.append(sieve[i])
        
        sieve = new_sieve
    
    return ludic
```


## inference on dev

In [6]:
import pandas as pd
import torch
from tqdm.auto import tqdm
import json

# Load dev set
df = pd.read_csv("/kaggle/input/blptask2data/gpt-translated.csv")  # Columns: id, instruction, test_list

# Disable truncation for strings
pd.set_option("display.max_colwidth", None)
df.head()

assert {"id", "instruction", "test_list"}.issubset(df.columns)

# df = df.sample(3)

# ================= Define system prompt =================
system_prompt = "You are a strict Python code generator. Follow the user instruction exactly. Never include any explanation or extra text."

# ================= Generate responses =================

# Generate responses
responses = []

for _, row in tqdm(df.iterrows(), total=len(df), desc="🔁 Generating"):
    full_prompt = (
        f"{guideline}\n\n"
        f"Here is the programming question: {row['instruction'].strip()}\n\n"
        f"Test cases:\n{row['test_list']}\n\n"
        f"Now write a Python code by following all the instructions to solve the given programming questions that covers all possible hidden test cases.\n"
    )

    response = client.messages.create(
        model="claude-sonnet-4-20250514",  # or "claude-3.5-sonnet-20240620"
        max_tokens=1024,
        temperature=0.3,
        system=system_prompt,
        messages=[
            {"role": "user", "content": full_prompt}
        ]
    )

    gen_text = response.content[0].text.strip()
    responses.append(gen_text)

# sanity check
assert len(df) == len(responses), "Mismatch between input and output rows!"

# Save
out_df = pd.DataFrame({"id": df["id"], "response": responses})
out_df.to_json("submission.json", orient="records", force_ascii=False, indent=2)
print(f"✅ Wrote submission.json with {len(out_df)} rows")

# Show top 10 entries from the saved JSON
with open("submission.json", "r", encoding="utf-8") as f:
    data = json.load(f)
    print("🔹 Top 10 JSON entries:")
    for entry in data[:10]:
        print(entry)


FileNotFoundError: [Errno 2] No such file or directory: '/kaggle/input/blptask2data/gpt-translated.csv'

In [ ]:
len(responses)

In [ ]:
out_df.shape

In [ ]:
df.shape

## Preparing Submission File

In [ ]:
import json, os, re, zipfile

SUB_PATH = "submission.json"

def file_format_check(path: str) -> bool:
    # name + extension
    if os.path.basename(path) != "submission.json":
        print("Error: File name must be exactly 'submission.json'")
        return False
    if not path.lower().endswith(".json"):
        print("Error: File must have .json extension")
        return False

    # must be valid JSON (not JSONL) and root must be a list
    try:
        with open(path, "r", encoding="utf-8") as f:
            data = json.load(f)
    except json.JSONDecodeError as e:
        print(f"Error: Invalid JSON format - {e}")
        print("Note: The file must be in proper JSON format (not JSONL)")
        return False

    if not isinstance(data, list):
        print("Error: The root element should be a list of objects")
        return False

    # each item: dict with ONLY keys {'id','response'}; id=int; response=str
    for idx, item in enumerate(data):
        if not isinstance(item, dict):
            print(f"Error: Item at index {idx} is not a dictionary")
            return False
        keys = set(item.keys())
        if keys != {"id", "response"}:
            print(f"Error: Item at index {idx} must contain only keys 'id' and 'response', found: {keys}")
            return False
        if not isinstance(item["id"], int):
            print(f"Error: 'id' field at index {idx} must be an integer")
            return False
        if not isinstance(item["response"], str):
            print(f"Error: 'response' field at index {idx} must be a string")
            return False

    print("Format check passed successfully!")
    return True

# ---------- Load, compute per-item validity, blank invalids, save, zip ----------
# Load JSON list
with open(SUB_PATH, "r", encoding="utf-8") as f:
    data = json.load(f)

n = len(data)
fence_pat = re.compile(r"^```python[\s\S]*```$", re.MULTILINE)

valid_format = []
valid_fence  = []
valid_both   = []

# Per-item validation mirrors file checker semantics
def item_format_ok(item):
    return (
        isinstance(item, dict)
        and set(item.keys()) == {"id", "response"}
        and isinstance(item["id"], int)
        and isinstance(item["response"], str)
    )

for item in data:
    vfmt = item_format_ok(item)
    vf   = bool(fence_pat.match(item["response"])) if vfmt else False
    valid_format.append(vfmt)
    valid_fence.append(vf)
    valid_both.append(vfmt and vf)

# After computing valid_format, valid_fence, valid_both
for i, item in enumerate(data):
    if not valid_format[i]:
        print(f"❌ Format Error at index {i}: {item}")
    elif not valid_fence[i]:
        print(f"❌ Fencing Error at index {i} (id={item.get('id')}):")
        print(item["response"])
        print("-" * 50)


# Report stats
nf = sum(valid_fence)
nm = sum(valid_format)
nb = sum(valid_both)
den = max(n, 1)
print(f"Fencing valid: {nf}/{n} ({nf*100.0/den:.1f}%)")
print(f"Format valid:  {nm}/{n} ({nm*100.0/den:.1f}%)")
print(f"Both valid:    {nb}/{n} ({nb*100.0/den:.1f}%)")

# Strict policy: blank responses that fail ANY check
for i, ok in enumerate(valid_both):
    if not ok and isinstance(data[i], dict) and "response" in data[i]:
        data[i]["response"] = ""

# Overwrite submission.json (id+response only)
with open(SUB_PATH, "w", encoding="utf-8") as f:
    json.dump(
        [{"id": item["id"], "response": item["response"]} for item in data],
        f, ensure_ascii=False, indent=2
    )
print("✅ Updated submission.json after checks (invalid responses blanked).")

# Final file-level check (should pass)
_ = file_format_check(SUB_PATH)

# Zip as submission.zip (Jupyter-friendly, no shell commands)
with zipfile.ZipFile("submission.zip", "w", compression=zipfile.ZIP_DEFLATED) as zf:
    zf.write(SUB_PATH)
print("📦 Created submission.zip containing submission.json.")
